In [2]:
import spacy
import re
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import nltk

# Download necessary NLTK data files
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load spaCy's English model
nlp = spacy.load("en_core_web_sm")

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

# List of stopwords
stop_words = set(stopwords.words("english"))

# Preprocessing function to clean and prepare the text
def preprocess_text(text):
    # Remove special characters, digits, and punctuations
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = text.lower()  # Convert text to lowercase

    # Tokenize the text
    words = word_tokenize(text)

    # Remove stopwords and lemmatize the words
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return " ".join(words)

# Function to extract preferences from reviews
def extract_preferences(reviews):
    preferences = []

    for review in reviews:
        # Step 1: Preprocess the review
        cleaned_review = preprocess_text(review)

        # Step 2: Apply spaCy NLP processing to extract noun chunks (food-related preferences)
        doc = nlp(cleaned_review)
        noun_chunks = [chunk.text for chunk in doc.noun_chunks]

        # Step 3: Sentiment Analysis using TextBlob
        blob = TextBlob(review)
        sentiment = blob.sentiment.polarity  # Range from -1 (negative) to 1 (positive)

        # Step 4: Categorize the feedback (Positive, Negative, Neutral)
        if sentiment > 0:
            sentiment_label = "Positive Feedback"
        elif sentiment < 0:
            sentiment_label = "Negative Feedback"
        else:
            sentiment_label = "Neutral Feedback"

        # Step 5: Capture extracted preferences and sentiment
        preferences.append({
            "review": review,
            "cleaned_review": cleaned_review,
            "noun_chunks": noun_chunks,
            "sentiment": sentiment_label
        })

    return preferences

# Example food-related reviews
reviews = [
    "The pizza was delicious, especially the crust which was crispy and flavorful.",
    "I hate the bland taste of the soup, it was too salty for my liking.",
    "The burger was juicy but the fries were soggy and undercooked.",
    "Absolutely loved the ice cream! So creamy and rich in flavor, will definitely buy again."
]

# Extract preferences
preferences = extract_preferences(reviews)

# Display extracted preferences
for i, pref in enumerate(preferences):
    print(f"\nReview {i + 1}: {pref['review']}")
    print(f"Cleaned Review: {pref['cleaned_review']}")
    print(f"Noun Phrases Extracted: {pref['noun_chunks']}")
    print(f"Sentiment: {pref['sentiment']}")



ModuleNotFoundError: No module named 'spacy'

In [ ]:
import dash
from dash import dcc, html, dash_table
import plotly.express as px
import pandas as pd
from jupyter_dash import JupyterDash  # Import JupyterDash instead of Dash


# Sample data with various cuisines
recipes = pd.DataFrame({
    "title": [
        "Beef Stroganoff", "Margherita Pizza", "Penne Arrabbiata",
        "Burritos", "Salsa", "Quesadillas",
        "General Tso's Chicken", "Mapo Tofu", "Egg Rolls",
        "Lamb Rogan Josh", "Palak Paneer", "Biryani"
    ],
    "ingredients": [
        "beef, mushrooms, sour cream", "pizza dough, mozzarella, basil", "penne, tomatoes, chili flakes",
        "tortillas, rice, beans, cheese", "tomatoes, onions, cilantro, lime", "tortillas, cheese, chicken",
        "chicken, soy sauce, hoisin sauce", "tofu, ground pork, chili bean paste", "egg roll wrappers, cabbage, pork",
        "lamb, yogurt, spices", "spinach, paneer, cream", "rice, chicken, saffron, spices"
    ],
    "directions": [
        "Cook beef with mushrooms, add sour cream", "Bake pizza dough with toppings", "Cook penne, mix with spicy tomato sauce",
        "Fill tortillas with rice, beans, and cheese", "Mix tomatoes, onions, cilantro, and lime", "Fill tortillas with cheese and chicken, cook",
        "Fry chicken, mix with sauce", "Cook tofu and pork with chili bean paste", "Fill wrappers with cabbage and pork, fry",
        "Cook lamb with yogurt and spices", "Cook spinach, mix with paneer and cream", "Cook rice with chicken and spices"
    ],
    "cuisine": [
        "Russian", "Italian", "Italian",
        "Mexican", "Mexican", "Mexican",
        "Chinese", "Chinese", "Chinese",
        "Indian", "Indian", "Indian"
    ]
})

# Bar chart for ingredient frequency
ingredient_counts = recipes['ingredients'].str.split(', ').explode().value_counts()
ingredient_bar_chart = px.bar(
    ingredient_counts,
    x=ingredient_counts.index,
    y=ingredient_counts.values,
    labels={'x': 'Ingredients', 'y': 'Frequency'},
    title='Frequency of Ingredients in Recipes'
)

# Pie chart for cuisine distribution
cuisine_pie_chart = px.pie(
    recipes,
    names='cuisine',
    title='Distribution of Cuisines in Recipes'
)

# Dash app setup
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Recipe Dashboard"),

    html.H2("Recipe Table"),
    dash_table.DataTable(
        data=recipes.to_dict('records'),
        columns=[{"name": col, "id": col} for col in recipes.columns],
        style_table={'overflowX': 'auto'}
    ),

    html.H2("Ingredient Frequency"),
    dcc.Graph(figure=ingredient_bar_chart),

    html.H2("Cuisine Distribution"),
    dcc.Graph(figure=cuisine_pie_chart),
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)